In [1]:
import re
import pandas as pd
import unicodedata

print(pd.__version__)
print(re.__version__)


2.2.3
2.2.1


In [2]:
df1 = pd.read_excel('학교별 교육편제단위 정보_20251002기준.xlsx', skiprows = 4 )
df2 = pd.read_excel('교육과정_대학(20251020).xlsx', skiprows = 8)

In [3]:
#교육과정과 학과학부 데이터 결합
df2 = df2.rename(columns={
    "차수" : "조사차수",
    "본분교명": "대학구분",
    "학교구분" : "본분교",
    "학부·과(전공)코드": "학교별학과코드",
    "주야구분명" : "주야간구분",
    "학부특성명" : "학과특성",

})

In [4]:
# 1️⃣ 키 컬럼 전처리 (공백/괄호 제거, 타입 통일)
for df in [df1, df2]:
    df['학교코드'] = df['학교코드'].astype(str).str.strip()
    df['학교별학과코드'] = df['학교별학과코드'].astype(str).str.strip()
    df['학부·과(전공)명'] = (
        df['학부·과(전공)명']
        .astype(str)
        .str.replace(r'\s+', '', regex=True)       # 모든 공백 제거
    )

# 2️⃣ df2에서 필요한 열만 선택
attach_cols = ['교육과정','이수구분','학점','교과목해설']
df2_slim = df2[['학교코드','학교별학과코드','학부·과(전공)명'] + attach_cols].copy()

# 3️⃣ 조인
merged = pd.merge(
    df1,
    df2_slim,
    on=['학교코드','학교별학과코드','학부·과(전공)명'],
    how='left',      # 학과정보(df1)를 기준으로
    validate='1:m'   # 한 학과에 여러 과목이 붙는 구조
)

print(len(merged))
merged


476878


,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,소계열분류,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설
0,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
1,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,N.C.E.,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
2,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
3,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
4,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476873,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476874,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,석사,서울,서초구,NaN,NaN,NaN,NaN
476875,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476876,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,N.C.E.,인문・사회계열,2년,석사,서울,서초구,NaN,NaN,NaN,NaN


In [50]:
merged.to_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\교육편제단위_표준분류체계 결합_v2.xlsx', index=False)

## 교육과정 정제
- 뒤에 숫자로 나뉘어져 있는 과목 통일(초급중국어1, 초급중국어2 -> 초급중국어)
- 한영 표기 동시에 되어 있는 과목 숫자 제거 후 한글 표현으로 통일 (ex. 한국어글쓰기와 표현1 (Korean writing and expression 1))

In [5]:
import re
import pandas as pd
import unicodedata


# 0️⃣ 전각→반각 정규화
def nfkc(x):
    return unicodedata.normalize('NFKC', str(x)) if pd.notna(x) else x


# 1️⃣ eng→kor 사전 생성
def build_eng2kor(series):
    eng2kor = {}
    norm = series.dropna().map(nfkc).unique().tolist()
    for c in norm:
        c_std = c.replace('（','(').replace('）',')')
        kor = re.sub(r'\(.*?\)', '', c_std).strip()
        inside = re.findall(r'\((.*?)\)', c_std)
        if inside:
            inside_text = inside[0].strip()
            if re.search(r'[A-Za-z]', inside_text):
                eng = re.sub(r'\d+([\-\.]\d+)?$', '', inside_text).strip()
                eng2kor[eng.lower()] = kor
    return eng2kor


# 2️⃣ 과목 정제 함수 (3D모델링 안전 버전)
def clean_course(name, eng2kor=None):
    if pd.isna(name):
        return None

    # (A) 전각→반각 정규화
    raw = nfkc(name).strip()

    # (B) 영어→한글 매핑
    if eng2kor:
        raw_lower = raw.lower()
        for eng, kor in eng2kor.items():
            if eng and eng in raw_lower:
                return kor  # 매핑 우선 적용

    # (C) 나머지 정제
    txt = raw
    
    # 공백 정리
    txt = re.sub(r'\s', '', txt)
    txt = re.sub(r'\[.*?\]', '', txt)  # 대괄호 제거

    # 중첩 괄호 제거 + 끝의 숫자 제거
    while re.search(r'\([^()]*\)', txt):
        txt = re.sub(r'\([^()]*\)', '', txt)
        # 끝 숫자 제거는 한글/영문자 바로 뒤 숫자만 대상으로 함
        txt = re.sub(r'(?<=[가-힣A-Za-z])[\-\.]?\d+$', '', txt)

    # 로마숫자 및 특수문자 제거
    txt = re.sub(r'[ｏ@#☆ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ]', '', txt)
    
    # 한글 뒤에 붙은 영문 로마숫자(I~X) 제거 (예: 건축설계스튜디오I → 건축설계스튜디오)
    txt = re.sub(r'(?<=[가-힣])(ll|l|I|II|III|IV|V|VI|VII|VIII|IX|X)$', '', txt)
        
    # 중간 숫자 제거 ('공업생화학1및실습' → '공업생화학및실습')
    txt = re.sub(r'(?<=[가-힣A-Za-z])\d+(?=[가-힣A-Za-z])', '', txt)

    # 끝의 숫자 제거 (한글·영문 뒤 숫자만, '3D모델링'은 그대로)
    txt = re.sub(r'(?<=[가-힣A-Za-z])\d+$', '', txt).strip()

    # 무의미한 문자열 처리
    if txt in ['', '/', '-', '--', 'NULL', 'NaN', '없음']:
        return None
    return txt


# 3️⃣ 숫자 버전 과목 통합 함수
def remove_number_if_duplicate(df, col='교육과정', verbose=True):
    """
    동일 과목명에서 숫자만 다른 경우를 통합
    (예: '공업생화학1및실습' + '공업생화학2및실습' → '공업생화학및실습')
    유일한 숫자 과목은 유지.
    """
    before = df[col].copy()

    # 비교용: 숫자 제거 버전
    df['비교용'] = df[col].str.replace(r'\d+', '', regex=True)

    # 숫자 제거 후 중복 존재하는 패턴만 식별
    dup_list = df['비교용'].value_counts()
    dup_list = dup_list[dup_list > 1].index.tolist()

    # 중복 패턴이면 숫자 제거, 아니면 그대로 둠
    df[col] = df.apply(
        lambda row: re.sub(r'\d+', '', row[col]) if row['비교용'] in dup_list else row[col],
        axis=1
    )

    # 임시 컬럼 삭제
    df.drop(columns='비교용', inplace=True)
    return df


In [64]:
# 0️⃣ 원본 백업
before_all = merged['교육과정'].copy()

# 1️⃣ 전각→반각 정규화만 비교
nfkc_step = merged['교육과정'].apply(nfkc)
print("전각→반각 변경:", (nfkc_step != merged['교육과정']).sum())

# 2️⃣ 영어→한글 매핑 + 정제
eng2kor = build_eng2kor(nfkc_step)
cleaned_step = nfkc_step.apply(lambda x: clean_course(x, eng2kor))
print("영문 매핑·특수문자·숫자 제거로 변경:", (cleaned_step != nfkc_step).sum())

# 3️⃣ 숫자 버전 통합
merged['교육과정'] = cleaned_step
merged = remove_number_if_duplicate(merged, col='교육과정')
print("숫자 중복 통합으로 변경:", (cleaned_step != merged['교육과정']).sum())

# ✅ 전체 단계 누적 변화
total_changed = (before_all != merged['교육과정']).sum()
print(f"전체 {len(merged):,}개 중 최종 명칭 변경된 행 수: {total_changed:,}개")

전각→반각 변경: 48364
영문 매핑·특수문자·숫자 제거로 변경: 48389
숫자 중복 통합으로 변경: 48364
전체 476,878개 중 최종 명칭 변경된 행 수: 48,389개


In [66]:
merged.to_excel('교육과정 변경내역.xlsx')

In [6]:
merged['표본수 n'] = (
    merged.groupby('교육과정')['교육과정'].transform('count').astype('Int64')
)

merged

,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설,표본수 n
0,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
1,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
2,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
3,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
4,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476873,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN,<NA>
476874,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,석사,서울,서초구,NaN,NaN,NaN,NaN,<NA>
476875,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN,<NA>
476876,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,2년,석사,서울,서초구,NaN,NaN,NaN,NaN,<NA>


In [68]:
merged.to_excel('과정 전체 데이터_20251022.xlsx')

In [69]:
nce_match = merged[(merged['대학구분'] == '대학') & (merged['학교구분'] == '대학교') & (merged['본분교'] == '본교') & (merged['(대학)지역'] == '서울')
          & (merged['주야간구분'] == '주간') & (merged['학과특성'] == '일반과정') & (merged['학과상태'] != '폐지') & (merged['수업연한'] == '4년')].copy() # 데이터 2070개 남음

In [70]:
nce_match.to_excel('nce매칭 필요 데이터_20251022.xlsx')